**Working on Claims severity :** Task is to create a better model and to find the predict the value for the 'loss' column. Variables prefaced with 'cat' are categorical, while those prefaced with 'cont' are continuous.Means must *predict the loss value for the ids in this file.*


# Here I am working with Tensorflow application **Regression model** named as DNNRegressor

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


For download the dataset click on the below link :
[link text](https://www.kaggle.com/c/allstate-claims-severity/data)

(By Jagannadan Manasa), Now i am trying to solve these Claims severity using a model.

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.logging.ERROR)

import warnings
warnings.filterwarnings("ignore")

In [0]:
train_o = pd.read_csv('/content/drive/My Drive/train.csv')
test_o = pd.read_csv('/content/drive/My Drive/test.csv')

In [21]:
train_df = train_o.head(1000)
evaluate_df = train_o.tail(500)

test_df = test_o.head(1000)

MODEL_DIR = "tf_model_full"

print("train_df.shape = ", train_df.shape)
print("evaluate_df.shape = ", evaluate_df.shape)
print("test_df.shape = ", test_df.shape)

train_df.shape =  (1000, 132)
evaluate_df.shape =  (500, 132)
test_df.shape =  (1000, 131)


In [0]:
features = train_df.columns
categorical_features = [feature for feature in features if 'cat' in feature]
continuous_features = [feature for feature in features if 'cont' in feature]
LABEL_COLUMN = 'loss'

# Convert the data in **Tensors**

In [0]:
def input_fn(df, training = True):
    continuous_cols = {k: tf.constant(df[k].values)
                       for k in continuous_features}
    categorical_cols = {k: tf.SparseTensor(
        indices=[[i, 0] for i in range(df[k].size)],
        values=df[k].values,
        dense_shape=[df[k].size, 1])
        for k in categorical_features}
    feature_cols = dict(list(continuous_cols.items()) +
                        list(categorical_cols.items()))

    if training:
        label = tf.constant(df[LABEL_COLUMN].values)
        return feature_cols, label    
    return feature_cols

def train_input_fn():
    return input_fn(train_df)

def eval_input_fn():
    return input_fn(evaluate_df)

def test_input_fn():
    return input_fn(test_df, False)

In [0]:
engineered_features = []

for continuous_feature in continuous_features:
    engineered_features.append(
        tf.contrib.layers.real_valued_column(continuous_feature))


for categorical_feature in categorical_features:
    sparse_column = tf.contrib.layers.sparse_column_with_hash_bucket(
        categorical_feature, hash_bucket_size=1000)

    engineered_features.append(tf.contrib.layers.embedding_column(sparse_id_column=sparse_column, dimension=16,
                                                                  combiner="sum"))

In [0]:
regressor = tf.contrib.learn.DNNRegressor(
    feature_columns=engineered_features, hidden_units=[10, 10], model_dir=MODEL_DIR)

# Fit the model

In [0]:
wrap = regressor.fit(input_fn = train_input_fn, steps=500)

In [43]:
print('Evaluating ...')
results = regressor.evaluate(input_fn=eval_input_fn, steps=1)
for key in sorted(results):
    print("%s: %s" % (key, results[key]))

Evaluating ...
global_step: 1000
loss: 9180887.0


In [47]:
predicted_output = regressor.predict(input_fn=test_input_fn)
set2_list = list(predicted_output )
print(list(set2_list[:10]))

[1368.6537, 2674.0889, 24399.498, 8121.3813, 885.21735, 4133.063, 1269.4844, 2564.876, 2470.7866, 2663.6309]


In [0]:
ID = df_test_ori['id']
df_test_ori.drop('id',axis=1,inplace=True)

# Save the results submission1.csv file

In [0]:
import numpy
with open("submission1.csv", "w") as subfile:
    subfile.write("id,loss\n")
    for i, pred in enumerate(list(set2_list)):
        subfile.write("%s,%s\n"%(ID[i],pred))